### installs and imports

In [1]:
pip install torch torchvision numpy matplotlib Pathlib -qqq


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


### Load Dataset from Kaggle

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("olgabelitskaya/horse-breeds")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'horse-breeds' dataset.
Path to dataset files: /kaggle/input/horse-breeds


### Split the data into train val, and test set (starified)

In [4]:
#first we list all image files and their labels, and split them
from pathlib import Path

dataset_path = Path(path)

image_paths = []
labels = []

for img_path in dataset_path.glob("*.png"):
    label = img_path.name.split("_")[0]
    
    image_paths.append(img_path)
    labels.append(label)
    print(f"Image: {img_path}, Label: {label}")


Image: /kaggle/input/horse-breeds/01_103.png, Label: 01
Image: /kaggle/input/horse-breeds/03_068.png, Label: 03
Image: /kaggle/input/horse-breeds/02_060.png, Label: 02
Image: /kaggle/input/horse-breeds/03_030.png, Label: 03
Image: /kaggle/input/horse-breeds/07_065.png, Label: 07
Image: /kaggle/input/horse-breeds/07_020.png, Label: 07
Image: /kaggle/input/horse-breeds/06_042.png, Label: 06
Image: /kaggle/input/horse-breeds/02_069.png, Label: 02
Image: /kaggle/input/horse-breeds/06_016.png, Label: 06
Image: /kaggle/input/horse-breeds/01_042.png, Label: 01
Image: /kaggle/input/horse-breeds/02_032.png, Label: 02
Image: /kaggle/input/horse-breeds/07_002.png, Label: 07
Image: /kaggle/input/horse-breeds/02_057.png, Label: 02
Image: /kaggle/input/horse-breeds/01_018.png, Label: 01
Image: /kaggle/input/horse-breeds/06_098.png, Label: 06
Image: /kaggle/input/horse-breeds/06_041.png, Label: 06
Image: /kaggle/input/horse-breeds/07_115.png, Label: 07
Image: /kaggle/input/horse-breeds/03_032.png, La

In [12]:
from sklearn.model_selection import train_test_split
train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42
)
print(f"training samples: {len(train_paths)}")
print(f"validation samples: {len(val_paths)}")


training samples: 536
validation samples: 134
